In [1]:
# necessary modules
import sqlite3
import utils
import recommender_classifier
import time
import numpy as np
# import matplotlib.pyplot as plt
from multiprocessing import Process, Manager

In [2]:
# load random users and feature vectors
conn = sqlite3.connect('database.db')
LOW_LEVEL_FEATURES, DEEP_FEATURES_RESNET, HYBRID_FEATURES_RESNET = utils.extract_features()
foo, DEEP_FEATURES_BOF, HYBRID_FEATURES_BOF = utils.extract_features('bof_128.bin')
# USER_TFIDF_FEATURES, MOVIE_TFIDF_FEATURES = utils.extract_tfidf_features()

In [ ]:
Users = utils.selectRandomUsers(conn, 0.0001)
print len(Users)
user_profiles_small = recommender_classifier.build_user_profiles(conn, Users)

In [ ]:
Users = utils.selectRandomUsers(conn, 0.01)
print len(Users)
user_profiles_average = recommender_classifier.build_user_profiles(conn, Users)

In [ ]:
Users = utils.selectRandomUsers(conn, 0.1)
print len(Users)
user_profiles_big = recommender_classifier.build_user_profiles(conn, Users)

In [ ]:
Users = utils.selectRandomUsers(conn, 1)
print len(Users)
user_profiles_full = recommender_classifier.build_user_profiles(conn, Users)

In [3]:
Users = utils.selectRandomUsers(conn, None)
print len(Users)


138445


In [5]:
print len(Users)/10


13844


In [6]:
tam = len(Users)/10
part1 = Users[:tam]
# part2 = Users[tam:tam*2]
# part3 = Users[:len(Users)/10]
# part4 = Users[:len(Users)/10]
# part5 = Users[:len(Users)/10]
# part6 = Users[:len(Users)/10]
# part7 = Users[:len(Users)/10]
# part8 = Users[:len(Users)/10]
# part9 = Users[:len(Users)/10]
# part10 = Users[:len(Users)/10]

users_profiles_part = recommender_classifier.build_user_profiles(conn, part1)

In [7]:
def run(user_profiles, N, feature_vector, feature_vector2=None):

    conn = sqlite3.connect('database.db')
    
    SumRecall, SumPrecision = 0, 0
    
    for user, profile in user_profiles.iteritems():                
        
        if feature_vector2 is not None:
            if np.sum(feature_vector[user]) == 0:
                print "Blank user profile", user
                continue
        
        hits = 0
        
        predictions = recommender_classifier.get_predict_collaborative_filtering(conn, profile, feature_vector, feature_vector2)
        # print "Predictions", sorted(predictions, key=lambda tup: tup[2], reverse=True)
        
        for elite_movie in profile['datasets']['elite_test']:
                        
            if feature_vector is list and elite_movie[0] not in feature_vector:
                continue
                
            # Predict to the user movie and to random movies that the user did not rated
            # print predictions            
            elite_prediction = recommender_classifier.get_prediction_elite(conn, elite_movie, profile, feature_vector, feature_vector2)
            all_predictions = predictions[:]
            all_predictions.append(elite_prediction)
            
            # print "Elite Movie", elite_movie, elite_prediction
            
            hits += recommender_classifier.count_hit(all_predictions, elite_movie, N)
        try:
            recall = hits / float(len(profile['datasets']['elite_test']))
            SumRecall += recall
            SumPrecision += (recall / float(N))
        except ZeroDivisionError:
            continue
        # print "Size is", len(predictions)
        # print "Predictions", sorted(predictions, key=lambda tup: tup[2], reverse=True)

    size = len(user_profiles)
    avgRecall = utils.evaluateAverage(SumRecall, size)
    avgPrecision = utils.evaluateAverage(SumPrecision, size)

    return avgPrecision, avgRecall

In [8]:
# import recommend_random

# def experiment(N, user_profiles_low_level, LOW_LEVEL_FEATURES, user_profiles_deep, DEEP_FEATURES):
def experiment(N, user_profiles, res_ll,res_random,res_deep_bof,res_hybrid_bof):
    
    global LOW_LEVEL_FEATURES, DEEP_FEATURES_BOF, HYBRID_FEATURES_BOF
    
    result = {}
    start = time.time()
    
    # Tag-based
    # p_t, r_t = run(user_profiles, N, USER_TFIDF_FEATURES, MOVIE_TFIDF_FEATURES)
    # print "Tag-based Recall", r_t, "Tag-based Precision", p_t, "For iteration with", N
    
    # LOW LEVEL FEATURES check precision, recall and mae
    p_l, r_l = run(user_profiles, N, LOW_LEVEL_FEATURES)
    res_ll[N] = {'ll': {'recall': r_l, 'precision': p_l} }
    print "Low-Level Recall", r_l, "Low-Level Precision", p_l, "For iteration with", N
        
    # DEEP FEATURES - BOF
    p_d, r_d = run(user_profiles, N, DEEP_FEATURES_BOF)
    res_deep_bof[N] = {'deep_bof': {'recall': r_d, 'precision': p_d} }
    print "Deep BOF Recall", r_d, "Deep BOF Precision", p_d, "For iteration with", N        
    
    # HYBRID - BOF
    p_h, r_h = run(user_profiles, N, HYBRID_FEATURES_BOF)
    res_hybrid_bof[N] = {'hybrid_bof': {'recall': r_h, 'precision': p_h}}
    print "Hybrid BOF Recall", r_h, "Hybrid BOF Precision", p_h, "For iteration with", N    

    p, r, mae = recommender_classifier.recommend_random(user_profiles, N)
    res_random[N] = {'random': {'recall': r, 'precision': p}}
    print "Random Recall", r, "Random Precision",  p, "Random MAE", mae, "For iteration with", N
    
    # return_dict[N] = {'ll': {'recall': r_l, 'precision': p_l}, 'deep': {'recall': r_d, 'precision': p_d}, 'hybrid': {'recall': r_h, 'precision': p_h}, 'random': {'recall': r, 'precision': p}}    
    end = time.time()
    print "Execution time", (end - start)
    
#     result = {'ll': {'recall': r_l, 'precision': p_l}, 'deep': {'recall': r_d, 'precision': p_d}, 'random': {'recall': r, 'precision': p}}
   

In [ ]:
user_profiles = users_profiles_part

In [ ]:
manager = Manager()
res_ll = manager.dict()
# res_deep = manager.dict()
# res_hybrid = manager.dict()
res_deep_bof = manager.dict()
res_hybrid_bof = manager.dict()
res_random = manager.dict()

In [10]:
iterations = range(1, 6)

jobs = []
for num in iterations:
    p = Process(target=experiment, args=(num,user_profiles, res_ll,res_random,res_deep_bof,res_hybrid_bof))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()
# print return_dict.values()

Process Process-5:
KeyboardInterrupt
Process Process-6:
Process Process-4:
Process Process-2:
Process Process-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
    self.run()
    self.run()
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/p

KeyboardInterrupt: 

    predictions = recommender_classifier.get_predict_collaborative_filtering(conn, profile, feature_vector, feature_vector2)
    predictions = recommender_classifier.get_predict_collaborative_filtering(conn, profile, feature_vector, feature_vector2)
    elite_prediction = recommender_classifier.get_prediction_elite(conn, elite_movie, profile, feature_vector, feature_vector2)
  File "recommender_classifier.py", line 296, in get_prediction_elite
    prediction = predictUserRating(conn, user_profile, elite_movie, feature_vector, feature_vector2)
  File "recommender_classifier.py", line 265, in predictUserRating
  File "recommender_classifier.py", line 315, in get_predict_collaborative_filtering
  File "recommender_classifier.py", line 315, in get_predict_collaborative_filtering
  File "recommender_classifier.py", line 315, in get_predict_collaborative_filtering
    prediction = predictUserRating(conn, user_profile, random_movie, feature_vector, feature_vector2)
    prediction = predictUse

In [ ]:
iterations = range(6, 11)

jobs = []
for num in iterations:
    p = Process(target=experiment, args=(num,user_profiles, res_ll,res_random,res_deep_bof,res_hybrid_bof))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

In [ ]:
low_level_recall = [item['ll']['recall'] for item in res_ll.values()]
# deep_recall = [item['deep_resnet']['recall'] for item in res_deep.values()]
# hybrid_recall = [item['hybrid_resnet']['recall'] for item in res_hybrid.values()]
deep_bof_recall = [item['deep_bof']['recall'] for item in res_deep_bof.values()]
hybrid_bof_recall = [item['hybrid_bof']['recall'] for item in res_hybrid_bof.values()]
random_recall = [item['random']['recall'] for item in res_random.values()]

# print low_level_recall
# print deep_recall
# print low_level_recall

# for key, value in res_ll.items():
#      print "Entry", key, value

with open('results_13844_users_6_to_10.txt', 'a') as resfile:
    for key, value in res_ll.items():
        resfile.write(str(key)+str(value)+"\n")
    for key, value in res_deep_bof.items():
        resfile.write(str(key) + str(value)+"\n")
    for key, value in res_hybrid_bof.items():
        resfile.write(str(key) + str(value)+"\n")
    for key, value in res_random.items():
        resfile.write(str(key) + str(value)+"\n")

In [ ]:
plt.plot(iterations, low_level_recall, 'r-', iterations, deep_recall, 'g-', iterations, hybrid_recall, 'b-', iterations, deep_bof_recall, 'c-', iterations, hybrid_bof_recall, 'k-', iterations, random_recall, 'y-')
plt.ylabel('Recall')
plt.xlabel('Iterations')
plt.show()

iterations = range(1, 6)
plt.plot(iterations, low_level_recall, 'r-', iterations, deep_recall, 'g-', iterations, hybrid_recall, 'b-', iterations, deep_bof_recall, 'c-', iterations, hybrid_bof_recall, 'k-', iterations, random_recall, 'y-')
plt.ylabel('Recall')
plt.xlabel('Iterations')
plt.show()

# fig, ax = plt.subplots()

# # Be sure to only pick integer tick locations.
# for axis in [ax.xaxis, ax.yaxis]:
#     axis.set_major_locator(ticker.MaxNLocator(integer=True))

# # Plot anything (note the non-integer min-max values)...
# x = np.linspace(-0.1, np.pi, 100)
# ax.plot(range(1,6), low_level_recall, 'r--', range(1,6), deep_recall, 'g--', range(1,6), hybrid_recall, 'b--', range(1,6), random_recall, 'y--')

# # Just for appearance's sake
# ax.margins(0.05)
# ax.axis('tight')
# fig.tight_layout()

# plt.show()
